## Installations

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
!sudo apt-get update && sudo apt-get install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,827 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,530 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-dri

In [3]:
pip install -U datasets[audio] librosa lightning torch torchvision torchaudio torchlibrosa museval torchcontrib --extra-index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 10.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.0/833.0 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 111.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 978.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00
   ━━

## Data Download

In [4]:
from dotenv import load_dotenv
import os
from huggingface_hub import login, snapshot_download
from datasets import load_dataset

repo_id = "kuross/dl-proj-classification"
dataset = load_dataset(repo_id)
train_dataset = dataset['train']


README.md:   0%|          | 0.00/479 [00:00<?, ?B/s]

data/train-00000-of-00008.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

data/train-00001-of-00008.parquet:   0%|          | 0.00/284M [00:00<?, ?B/s]

data/train-00002-of-00008.parquet:   0%|          | 0.00/151M [00:00<?, ?B/s]

data/train-00003-of-00008.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

data/train-00004-of-00008.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

data/train-00005-of-00008.parquet:   0%|          | 0.00/30.1M [00:00<?, ?B/s]

data/train-00006-of-00008.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

data/train-00007-of-00008.parquet:   0%|          | 0.00/429M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5276 [00:00<?, ? examples/s]

In [5]:
from datasets import load_dataset, DatasetDict

repo_id = "kuross/dl-proj-classification"
dataset = load_dataset(repo_id)

# Start with the original 'train' split
original_train_dataset = dataset['train']

# --- Step 1: Split into Temporary Train/Validation and Test ---

# Allocate a portion (e.g., 10%) for the final TEST set
temp_splits = original_train_dataset.train_test_split(
    test_size=0.10, # 10% for the final test set
    seed=42
)

test_dataset = temp_splits['test']          # 10% of the original data
temp_train_val = temp_splits['train']       # 90% of the original data

# --- Step 2: Split the remaining data into Train and Validation ---

# Split the 90% remaining data (temp_train_val) into 80% Train and 10% Validation
# We calculate the ratio relative to the remaining data (90%)
# If we want 10% of the total (10 / 90) = 1/9 ≈ 0.1111
val_ratio_of_temp = 0.1111 

final_splits = temp_train_val.train_test_split(
    test_size=val_ratio_of_temp, # This is approximately 1/9th of the remaining data
    seed=42
)

train_dataset = final_splits['train']       # ~80% of the total data
val_dataset = final_splits['test']          # ~10% of the total data

# --- Final Dataset Structure ---

dataset = DatasetDict({
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
})

# --- Verification ---
print("--- Split Verification (Approximate 80/10/10 Split) ---")
total_size = len(original_train_dataset)
print(f"Total size: {total_size} samples")
print("-" * 25)
print(f"Train size: {len(dataset['train'])} ({len(dataset['train'])/total_size:.2%})")
print(f"Validation size: {len(dataset['val'])} ({len(dataset['val'])/total_size:.2%})")
print(f"Test size: {len(dataset['test'])} ({len(dataset['test'])/total_size:.2%})")
print("-" * 25)
print(f"New Dataset Structure: {dataset}")

--- Split Verification (Approximate 80/10/10 Split) ---
Total size: 5276 samples
-------------------------
Train size: 4220 (79.98%)
Validation size: 528 (10.01%)
Test size: 528 (10.01%)
-------------------------
New Dataset Structure: DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'duration'],
        num_rows: 4220
    })
    val: Dataset({
        features: ['file', 'label', 'duration'],
        num_rows: 528
    })
    test: Dataset({
        features: ['file', 'label', 'duration'],
        num_rows: 528
    })
})


## Configuration

In [87]:
# Using Base from HTS AT Repository
from external.hts_audio_transformer import config

## Add / Modify Configurations
config.debug = True
config.waveform_length = 128000
config.max_epoch = 5
config.classes_num = 5

## Data Preparation

### Resampling to 32000 Hz

In [88]:
train_dataset[0]

{'file': <datasets.features._torchcodec.AudioDecoder at 0x7abf2ebfdc90>,
 'label': 0,
 'duration': 0.0}

In [89]:
from datasets import Audio, Dataset


def resample_audio(dataset: Dataset, sampling_rate: int = 32000, audio_column_name: str = 'file'):
    resampled_dataset = dataset.cast_column(
        audio_column_name,
        Audio(sampling_rate=sampling_rate)
    )

    print(f"Dataset's audio has been resampled to {sampling_rate} Hz")

    return resampled_dataset

resampled_train_dataset = resample_audio(train_dataset)
resampled_val_dataset = resample_audio(val_dataset)
resampled_test_dataset = resample_audio(test_dataset)

Dataset's audio has been resampled to 32000 Hz
Dataset's audio has been resampled to 32000 Hz
Dataset's audio has been resampled to 32000 Hz


### Formatting for Dataset

In [90]:
from datasets import Dataset
import numpy as np


def format_dataset(dataset: Dataset, audio_column_name: str = 'file'):

    def format_function(record, idx):
        audio = record[audio_column_name]

        return {
            "audio_name": str(idx),
            "target": record['label'],
            "waveform": audio['array'],
            "real_len": len(audio['array'])
        }

    formatted_dataset = dataset.map(
        format_function,
        with_indices=True,
        remove_columns=[audio_column_name, 'label', 'duration']
    ).with_format("torch")

    return formatted_dataset


formatted_train_dataset = format_dataset(resampled_train_dataset)
formatted_val_dataset = format_dataset(resampled_val_dataset)
formatted_test_dataset = format_dataset(resampled_test_dataset)

In [91]:
# # For saving the formatting
# SAVE_PATH = "./formatted_audio_data"
# formatted_train_dataset.save_to_disk(SAVE_PATH)

# print(f"Dataset successfully saved to: {SAVE_PATH}")

In [92]:
# # Loading
# from datasets import load_from_disk
# import torch

# SAVE_PATH = "./formatted_audio_data"

# # 1. Load the dataset from the disk directory
# loaded_dataset = load_from_disk(SAVE_PATH)

# # 2. Re-apply the PyTorch format for training compatibility
# # This ensures it yields torch.Tensors when iterated over.
# loaded_dataset_torch = loaded_dataset.with_format("torch")

# print(f"Dataset successfully loaded from: {SAVE_PATH}")
# print(f"Loaded dataset object type: {type(loaded_dataset_torch)}")

In [93]:
# import os

# NUM_CLASSES = 527

# def format_audio_data_with_index(example, idx):
#     """
#     Formats the audio data, using the dataset index as the 'name'.
    
#     Args:
#         example (dict): The dictionary containing the row data.
#         idx (int): The index of the row in the dataset.
#     """
#     # Assuming the audio has been correctly decoded by the map function
#     audio_info = example[AUDIO_COLUMN_NAME]

#     # Get the single integer label
#     label_int = example['label']
    
#     # 1. Create a NumPy array of zeros with shape (NUM_CLASSES,)
#     one_hot_vector = np.zeros(NUM_CLASSES, dtype=np.float32)
    
#     # 2. Set the index corresponding to the label to 1.0
#     one_hot_vector[label_int] = 1.0

#     return {
#         "audio_name": str(idx),
#         "target": one_hot_vector,
#         "waveform": audio_info['array'],
#         "real_len": len(audio_info['array'])
#     }

# resampled_train_dataset = resampled_train_dataset.select(range(1000))

# formatted_train_dataset = resampled_train_dataset.map(
#     format_audio_data_with_index,
#     with_indices=True,
#     remove_columns=[AUDIO_COLUMN_NAME, 'label', 'duration']
# ).with_format("torch")

### Create Dataloader

In [94]:
import torch
import torch.nn.functional as F

def ensure_fixed_waveform_length(waveform: torch.Tensor, target_length: int = 128000) -> torch.Tensor:
    """
    Ensures an audio waveform tensor is exactly the target_length by cropping the start 
    or zero-padding the end, using optimized PyTorch operations.

    Args:
        waveform (torch.Tensor): The input audio time series tensor (e.g., shape [L]).
        target_length (int): The required length for the output waveform.

    Returns:
        torch.Tensor: The processed waveform with the exact target_length.
    """
    original_length = waveform.size(0)

    if original_length > target_length:
        return waveform[:target_length]
        
    elif original_length < target_length:
        padding_amount = target_length - original_length
        padding_tuple = (0, padding_amount)

        return F.pad(waveform, padding_tuple, mode='constant', value=0.0)
        
    else:
        return waveform


In [95]:
from torch.utils.data import Dataset
import logging
import random

class HTSATDataset(Dataset):
    def __init__(self, dataset, config, eval_mode = False):
        self.dataset = dataset
        self.config = config       
        self.total_size = len(self.dataset)
        self.queue = [*range(self.total_size)]
        logging.info("total dataset size: %d" %(self.total_size))
        if not eval_mode:
            self.generate_queue()

    def generate_queue(self):
        random.shuffle(self.queue)
        logging.info("queue regenerated:%s" %(self.queue[-5:]))


    def __getitem__(self, index):
        # Get actual index of the record
        p = self.queue[index]

        # Process waveform into fixed length
        waveform_processed = ensure_fixed_waveform_length(self.dataset[p]["waveform"], config.waveform_length)

        # one-hot target if required
        if config.loss_type == 'clip_bce':
            label_tensor = torch.as_tensor(self.dataset[p]["target"], dtype=torch.long)
            target = F.one_hot(label_tensor, num_classes=config.classes_num).float()
        else:
            target = torch.as_tensor(self.dataset[p]["target"])

        data_dict = {
            "audio_name": self.dataset[p]["audio_name"],
            "waveform": waveform_processed,
            "real_len": len(waveform_processed),
            "target": target
        }
        logging.info(f"getitem: {p}")
        return data_dict

    def __len__(self):
        return self.total_size


htsat_train_dataset = HTSATDataset(formatted_train_dataset, config)
htsat_val_dataset = HTSATDataset(formatted_val_dataset, config, True)
htsat_test_dataset = HTSATDataset(formatted_test_dataset, config, True)

In [ ]:
import lightning as L
import torch.optim as optim
import bisect
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score

from external.hts_audio_transformer.utils import get_loss_func


class HTSATModel(L.LightningModule):
    def __init__(self, model, config):
        super().__init__()
        self.model = model
        self.config = config
        # self.dataset = dataset
        self.loss_func = get_loss_func(config.loss_type)
        self.test_step_outputs = []
    
    def forward(self, x, mix_lambda = None):
        output_dict = self.model(x)
        return output_dict["clipwise_output"] , output_dict["framewise_output"]
    
    def training_step(self, batch, batch_idx):
        self.device_type = next(self.parameters()).device
        mix_lambda = None
        
        pred, _ = self(batch["waveform"], mix_lambda)
        loss = self.loss_func(pred, batch["target"])
        self.log("loss", loss, on_epoch= True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr = self.config.learning_rate, 
            betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0.05, 
        )
        # Change: SWA, deprecated
        # optimizer = SWA(optimizer, swa_start=10, swa_freq=5)
        def lr_foo(epoch):       
            if epoch < 3:
                # warm up lr
                lr_scale = self.config.lr_rate[epoch]
            else:
                # warmup schedule
                lr_pos = int(-1 - bisect.bisect_left(self.config.lr_scheduler_epoch, epoch))
                if lr_pos < -3:
                    lr_scale = max(self.config.lr_rate[0] * (0.98 ** epoch), 0.03 )
                else:
                    lr_scale = self.config.lr_rate[lr_pos]
            return lr_scale
        scheduler = optim.lr_scheduler.LambdaLR(
            optimizer,
            lr_lambda=lr_foo
        )
        
        return [optimizer], [scheduler]
    
    def validation_step(self, batch, batch_idx):
        pred, _ = self(batch["waveform"])
        return [pred.detach(), batch["target"].detach()]
    
    def time_shifting(self, x, shift_len):
        shift_len = int(shift_len)
        new_sample = torch.cat([x[:, shift_len:], x[:, :shift_len]], axis = 1)
        return new_sample 
    
    def test_step(self, batch, batch_idx):
        pred, pred_map = self(batch["waveform"])    # pred: (B, C) | pred_map: (B, window_size, C)
        target = batch['target']                    # target: (B, C)
        loss = self.loss_func(pred, target)

        output = {
            'pred': pred.detach().cpu(), 
            'target': target.detach().cpu(),
        }
        self.test_step_outputs.append(output)

        return
    
    def evaluate_metric(self, pred, target):
        # Convert to Numpy for use
        pred_numpy = pred.numpy()
        target_numpy = target.numpy()

        # Accuracy
        predicted_labels = np.argmax(pred_numpy, axis=1)
        true_labels = np.argmax(target_numpy, axis=1)
        acc = accuracy_score(true_labels, predicted_labels)

        # Loss
        loss = self.loss_func(pred, target).item()

        # mAP
        mAP = average_precision_score(target_numpy, pred_numpy, average='macro')

        return {
            "acc": acc,
            "loss": loss,
            "mAP": mAP
        }

    def on_test_epoch_end(self):
        test_step_outputs = self.test_step_outputs

        all_preds = torch.cat([x['pred'] for x in test_step_outputs], dim=0)
        all_targets = torch.cat([x['target'] for x in test_step_outputs], dim=0)
    
        metric_results = self.evaluate_metric(all_preds, all_targets)

        self.log_dict(metric_results,  on_epoch=True, prog_bar=True, sync_dist=True)

    

In [97]:
minimal_trainer = L.Trainer(
    # --- Essentials ---
    max_epochs=config.max_epoch,         # Stop condition for training
    default_root_dir="./checkpoints",     # Where logs and checkpoints go

    # --- Minimal Device Configuration (Assuming single GPU/CPU) ---
    # Setting devices=1 for modern Lightning (replaces gpus=1)
    # If device_num is 0 or 1, you can often just omit this
    # to let Lightning detect the device, but specifying is safer.
    # Note: If you need to specify CPU/GPU, use: devices=1, accelerator='gpu'/'cpu'
    
    # Keeping 'gpus' for compatibility with older PL versions in your original code:
    # gpus=device_num, 
    # log_every_n_steps = 1
)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores


In [101]:
from external.hts_audio_transformer.model.htsat import HTSAT_Swin_Transformer

sed_model = HTSAT_Swin_Transformer(
    num_classes=config.classes_num,
    config = config
)
model = HTSATModel(sed_model, config)

In [102]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset = htsat_train_dataset,
    num_workers = config.num_workers,
    # num_workers = 0,
    batch_size = 10,
    shuffle = False,
    # sampler = train_sampler
)


val_loader = DataLoader(
    dataset = htsat_val_dataset,
    num_workers = config.num_workers,
    # num_workers = 0,
    batch_size = 10,
    shuffle = False,
    # sampler = train_sampler
)

test_loader = DataLoader(
    dataset = htsat_test_dataset,
    num_workers = config.num_workers,
    # num_workers = 0,
    batch_size = 10,
    shuffle = False,
    # sampler = train_sampler
)

In [103]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
minimal_trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type                   | Params | Mode 
---------------------------------------------------------
0 | model | HTSAT_Swin_Transformer | 28.6 M | train
---------------------------------------------------------
27.6 M    Trainable params
1.1 M     Non-trainable params
28.6 M    Total params
114.564   Total estimated model params size (MB)
217       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.


In [104]:
trained_model = model.model
model = HTSATModel(trained_model, config)

In [106]:
test_results = minimal_trainer.test(
    model, 
    dataloaders=test_loader
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            acc            │     0.939393937587738     │
│           loss            │    0.05973969027400017    │
│            mAP            │    0.7804883122444153     │
└───────────────────────────┴───────────────────────────┘